This Notebook is a revision of the previously generated AI description of **Partnership Agreements** in 01_main from COARs to provide a more complete overview of partnerships and collaborations.

In [ ]:
# IMPORTANT: The parameters below are set only for running this notebook independently. 
# When executing the full Ploomber pipeline, these values will be overridden by the settings in `pipeline.yaml`. 
# Any modifications made here will not persist when running the pipeline.
COUNTRY =  'ARM' # Code of the Country
upstream = {
    "100_coar_sections": {
        "data": f"../data/processed/{COUNTRY}/COAR_Sections.xlsx",
    }
}
product = {
    "data": f"../data/processed/{COUNTRY}/101_coar_partnerships_and_collaborations.xlsx",
}

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from unicef_cpe.config import PROJ_ROOT
import unicef_cpe

In [ ]:
# other settings
MODEL = 'gpt-4o'
country_map = {k:v for k,v in unicef_cpe.utils.get_ecaro_countries_mapping(priority=False).items() if k in COUNTRY}
country_code_map = {v:k for k,v in country_map.items() }

In [ ]:
df_coars = pd.read_excel(upstream['100_coar_sections']['data'])

In [ ]:
# Restrict by filtering only countries under evaluation
country_mask = df_coars['country'].eq(COUNTRY)
df_coars = df_coars[country_mask].copy()


In [ ]:
#TODO!**NOTE** Collaboration and Other Partnerships are available as paragraphs only since 2020. 

In [ ]:
# Filter only section relatives to partnerships and collaborations
mask = df_coars['section'].eq('UN Collaboration and Other Partnerships')
df_coars = df_coars[mask].copy()

In [ ]:
df_coars['year_text'] = df_coars['year'].astype(str) + ' ' + df_coars['text']

In [ ]:
df_coars = df_coars.sort_values(by=['country', 'year'])
df_coars = df_coars.groupby('country')['year_text'].agg('\n'.join).reset_index(name='text')

In [ ]:

for index, row in tqdm(df_coars.iterrows()):
    summary = unicef_cpe.genai.summarise_coar_partnerships_v2(row['text'][:110_000], country_map.get(row['country']), model=MODEL)
    df_coars.loc[index, 'partnerships_summary'] = summary


In [ ]:
print('Shape:', df_coars.shape)
print(df_coars.head())

In [ ]:
output_path = Path(product['data'])
output_path.parent.mkdir(parents=True, exist_ok=True)  # Create missing directories
df_coars.to_excel(product['data'], index=False)

In [ ]:
################################################################################################################################################################################################